In [2]:
!pip install geopy
!pip install folium

     |████████████████████████████████| 94 kB 6.3 MB/s  eta 0:00:01


In [3]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
address = 'mumbai,IN'

geolocator = Nominatim(user_agent = "project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai(Maharashtra),India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai(Maharashtra),India are 19.0759899, 72.8773928.


In [5]:
neighborhood_latitude=19.0759899
neighborhood_longitude=72.8773928

In [6]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = '01ZMVXTFVOPHFISXKR35CSTSDBCM2XLKVVSBUMJO1XGQKGYD' # your Foursquare ID
CLIENT_SECRET = 'PJYJP2NFYNMMKCYQN11JKOJX2VWYS54YZ2SGRULTZVYER2IB' # your Foursquare Secret
VERSION = '20180604'
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=01ZMVXTFVOPHFISXKR35CSTSDBCM2XLKVVSBUMJO1XGQKGYD&client_secret=PJYJP2NFYNMMKCYQN11JKOJX2VWYS54YZ2SGRULTZVYER2IB&v=20180604&ll=19.0759899,72.8773928&radius=500&limit=100'

In [7]:
results = requests.get(url).json()

In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Delhi Zaika,Indian Restaurant,19.077054,72.878260
1,Pizza Hut,Pizza Place,19.075984,72.877656
2,Nawab Sheek Corner,Middle Eastern Restaurant,19.076933,72.878260
3,Sahara Restaurant,Mughlai Restaurant,19.079532,72.880152
4,Vigour gym,Gym,19.075981,72.877656
5,kurla market,Flea Market,19.079207,72.880212
6,Manoj medical,Pharmacy,19.074811,72.872805


In [11]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=17)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_mum)  
    
map_mum